# Lista 7 - Atenção Neural

Nesta lista exploraremos o mecanismo de atenção neural. Os mecanismos de atenção foram originalmente propostos como uma forma de incrementar os modelos recorrentes de tradução automática de textos, adicionando um mecanismo que permitiria o alinhamento entre os elementos do texto de origem e os elementos do texto de saída.

Após as propostas iniciais, os mecanismos de atenção neural tiveram seu escopo expandido, assumindo um lugar de protagonismo em vários dos avanços que se sucederaam no NLP e no aprendizado de máquina, como um todo.

Esses avanços posteriores serão discutidos nas próximas semanas, quando falarmos sobre a atenção neural no contexto dos Transformers. Nessa lista abordaremos a atenção neural sob uma ótica similiar àquela de sua proposição inicial, como um  mecanismo associado às redes neurais recorrentes.

In [17]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential

## Dados

Para explorarmos a ação dos mecanismos de atenção quando acoplados a redes recorrentes usaremos um problema simplificado: prever qual o próximo termo da [sequência de Fibonacci](https://en.wikipedia.org/wiki/Fibonacci_sequence) a partir de uma janela dos $t$ termos anteriores.

Abaixo disponibilizamos o código, baseado [neste material](https://machinelearningmastery.com/adding-a-custom-attention-layer-to-recurrent-neural-network-in-keras/), que gera os números da sequência de Fibonacci na forma necessária para o treinamento e avaliação dos nossos modelos. O Código gera um conjunto de treinamento (`trainX` e `trainY`) e um conjunto de testes (`testX` e `testY`). Os exemplos de cada conjunto são janelas de $t$ termos sequenciais da sequência de Fibonacci com o termo seguinte como resposta. Os conjuntos são embaralhados e os números normalizados para facilitar o aprendizado da rede.
Obs: Nos nossos experimentos, usaremos $t=20$

In [18]:
def get_fib_seq(n, scale_data=True):
    # Get the Fibonacci sequence
    seq = np.zeros(n)
    fib_n1 = 0.0
    fib_n = 1.0
    for i in range(n):
            seq[i] = fib_n1 + fib_n
            fib_n1 = fib_n
            fib_n = seq[i]
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        seq = np.reshape(seq, (n, 1))
        seq = scaler.fit_transform(seq).flatten()
    return seq, scaler

def get_fib_XY(total_fib_numbers, time_steps, train_percent, scale_data=True):
    dat, scaler = get_fib_seq(total_fib_numbers, scale_data)
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
        X = np.column_stack((X, temp))
    # random permutation with fixed seed
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler


trainX, trainY, testX, testY, scaler  = get_fib_XY(1200, 20, 0.7)

# <font color='blue'>  Questão 1 </font>

Explique, em poucas palavras, porque esse é um problema apropriado para ilustrar a aplicação de mecanismos de atenção às redes recorrentes. Se precisar, revise o conteúdo das últimas aulas, o texto de abertura dessa lista e a definição da sequência de de Fibonacci.

_______________________________________________

**<font color='red'> Sua resposta aqui </font>**

O mecanismo de atenção neural tem o propósito de identificar a parte mais relevante dentro de um contexto e medir a correlação entre a entrada e a saída. Na sequência de Fibonacci, a predição do elemento seguinte depende apenas dos dois elementos anteriores e não da sequência completa. Assim, nesse contexto, o mecanismo de atenção deve ser capaz de identificar que apenas esses elementos são relevantes e que o restante da sequência pode ser desconsiderado na predição.

## Rede Neural Recorrente sem Atenção

Nosso objetivo final é comparar, nessa tarefa simplificada, a performance de uma rede neural recorrente sem o mecanismo de atenção a uma rede neural recorrente com mecanismo de atenção.Vamos começar pela versão sem mecanismo de atenção.



# <font color='blue'>  Questão 2 </font>

Complete o código a seguir de de forma que o modelo implementado seja constituído de uma camada de Rede Neural recorrente básica com duas unidades escondidas, seguinda de uma camada densa com uma unidade. Para ambas as camadas use a Tangente Hibperbólica como função de ativação
Obs: para a camada recorrente use a camada pré-implementada [Simple RNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/), do Keras.

In [20]:
input_shape = (20, 1)
model_vanilla = Sequential()

# Seu código começa aqui

model_vanilla.add(Input(shape=input_shape))
model_vanilla.add(SimpleRNN(2, return_sequences=True, activation='tanh'))
model_vanilla.add(Dense(1, activation='tanh'))

# Seu código termina aqui

model_vanilla.compile(loss='mse', optimizer='adam')

model_vanilla.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)
train_mse = model_vanilla.evaluate(trainX, trainY)
test_mse = model_vanilla.evaluate(testX, testY)
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)


Epoch 1/30
826/826 - 3s - loss: 0.0014 - 3s/epoch - 4ms/step
Epoch 2/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 3/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 4/30
826/826 - 3s - loss: 0.0013 - 3s/epoch - 4ms/step
Epoch 5/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 6/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 7/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 8/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 9/30
826/826 - 3s - loss: 0.0013 - 3s/epoch - 4ms/step
Epoch 10/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 11/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 12/30
826/826 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 13/30
826/826 - 2s - loss: 0.0012 - 2s/epoch - 3ms/step
Epoch 14/30
826/826 - 3s - loss: 0.0012 - 3s/epoch - 4ms/step
Epoch 15/30
826/826 - 2s - loss: 0.0012 - 2s/epoch - 3ms/step
Epoch 16/30
826/826 - 2s - loss: 0.0012 - 2s/epoch - 3ms/step
Epoch 17/30
826/8

# Rede Neural recorrente com Atenção

Agora vamos implementar a versão com atenção. Para tal implementaremos uma camada de atenção personalizada.

Um dos mecanismos de atenção mais simples que existem pode ser definido através das seguintes equações:

$E = tanh(X \cdot W + b)$

$\alpha = softmax(E)$

$X^\prime = \alpha X$

Onde $X$ é a matriz que aglomera representações sequenciais de vários inputs, $\alpha$ é a matriz com os pesos de atenção e $X^\prime$ é a saída, uma nova representação para os inputs, pesada pelos fatores de atenção calculados. A matriz $W$ e o vetor $b$ são parâmetros internos da camada de atenção que serão otimizados no treinamento.
# <font color='blue'>  Questão 3 </font>
O código dado a seguir implementa uma camada de atenção personalizada seguindo o formato descrito até aqui, em seguida, constrói um modelo com essa camada de atenção, treina e testa nos dados. Complete as lacunas de maneira  a implementrar o mecanismo de atenção aqui descrito. Use as funções do módulo `keras.backend`, importado no início desta lista.

In [21]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Descomente e complete a linha a seguir:
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        # Descomente e complete a linha a seguir:
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')
    return model

# Create the model with attention, train and evaluate
model_attention = create_RNN_with_attention(hidden_units=2, dense_units=1,
                                  input_shape=(20,1), activation='tanh')
model_attention.summary()


model_attention.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)

# Evalute model
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)

# Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 20, 1)]           0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 20, 2)             8         
                                                                 
 attention_6 (attention)     (None, 2)                 22        
                                                                 
 dense_6 (Dense)             (None, 1)                 3         
                                                                 
Total params: 33 (132.00 Byte)
Trainable params: 33 (132.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
826/826 - 4s - loss: 0.0019 - 4s/epoch - 5ms/step
Epoch 2/30
826/826 - 2s - loss: 0.0018 - 2s/epoch - 3ms/step
Epoch 3/30
826/826 - 2s - loss: 0